<div class="jumbotron m-0">
    <hr />
    <h1 class="text-center">
        <span class="text-primary">ROS 2 Navigation in 5 Days
        </span>
    </h1>
    <hr />
</div>

<img src="images/ros2_nav_details.png"/>

<div class="bg-primary text-center">
    - Summary -
</div>

In this rosject, you will apply everything you've learned in the ROS2 Navigation course to a TurtleBot3 real robot.

You will practice with a Turtlebot3 robot, in both **the simulation** and **the real one**. 

* You will launch the simulation here, in this rosject
* You will connect to the real robot remotely, also from this rosject

More instructions below.

<div class="bg-primary text-center">
    - End of Summary -
</div>

![sima_and_real](images/sim_and_real_lab.png)

Things to remember:
* Create your packages and code in the simulation environment as you have been doing throughout the course.
* Use the consoles to gather information about the status of the simulated and real robot.
* Use the IDE to create your files and execute them through the consoles, observing the results on the simulation screen. You can use other consoles to watch calls to topics, services, or action servers.
* Everything that you create in this unit will be automatically saved in your space. You can come back to this unit at any time and continue with your work from the point where you left off.
* Every time you need to reset the position of the robot, just press the restart button in the simulation window. 
* Use the debugging tools to try to find what is not working and why (for instance, the Rviz tool is very useful for this purpose).

<div>
    <h1 class="text-center">
        <span class="text-primary">0</span>
        &nbsp;
        <span class="">Launch the simulation</span>
    </h1>
</div>

You should use the simulation to test your code while you try to get the project's exercises done.

To launch the simulation, do the following:

1. Open a terminal by pressing on the *terminal icon* on the bottom left side of the screen:
<img src="images/open_terminal_superapp.png"/>
2. Type the following command that launches the simulation

In [ ]:
source ~/simulation_ws/install/setup.bash
ros2 launch turtlebot3_gazebo main_turtlebot3_lab.launch.xml

3. To see the simulation, press the Gazebo button on the bottom left side of the screen, and select *Open gazebo*:
<img src="images/gazebo_superapp.png"/>

**Wait about 30 seconds maximum** for the simulation to appear in the Gazebo window.

<img src="images/new_sim.png"/>


## How this rosject works

### TurtleBot3 Topics

**Sensors**
* Laser sensor: This TurtleBot3 has an RPLIDAR, which provides information about the objects that it detects in its range. The value of the sensor is provided through the topic `/scan`.
* Odometry: The odometry of the robot can be accessed through the `/odom` topic.

**Actuators**
* Speed: You can send speed commands to move the robot through the topic `/cmd_vel`.

## How to Proceed

This rosject is composed of five sections and an Appendix, explaining how to connect to the real robot. You should do the first four sections when the ROS Navigation in 5 Days course indicates it. These steps will ensure that you have practiced and created all of the structures asked about in the final presentation of this course. 

* **SECTION I:** Create a map of the environment
* **SECTION II:** Localize the robot
* **SECTION III:** Create a Path Planning System
* **SECTION IV:** Create a ROS program that interacts with the Navigation Stack
* **SECTION V:** Book a date to present the rosject on YouTube and share it on LinkedIn or social networks (Optional)

Go to the section that the course indicates and try to finish it. Before doing that, **launch the rosject simulation**. You will use this simulation to practice with the simulated environment.

<div>
    <h1 class="text-center">
        <span class="text-primary">Section 1</span>
        &nbsp;
        <span class="">Mapping
    </h1>
</div>

As you've learned in the first 2 chapters of the course, the first thing you need in order to navigate autonomously with a robot is a map of the environment. Where else would this project start?

In this section, you will practice how to map the environment around the robot. Your goal is to save this map file in order to use it for localization and navigation.

We divide this into 7 steps:

1. Make sure that the TurtleBot3 robot is publishing its **transform data** correctly.
2. Create a package called `/project_mapping` that will contain all of the files related to mapping.
3. Create a launch file that will launch the `cartographer_node` and add the necessary **parameters** in order to properly configure the slam_gmapping node.
4. Launch the `cartographer_node` and **create a map** of the simulated environment while tele-operating the robot.
5. **Save the map** that you've just created by creating a launch file for the `map_saver` node.
7. Create a **launch file** that will provide the created map to other nodes.

So, let's go!

**1. Generate Files to Visualize the TF Frame Tree**

You should get a TF tree like this one:

<img src="images/turtlebot_tf_tree.png"/> 

Pay close attention to the important frames, you'll use them later.

**2. Create a package called `project_mapping` in `ros2_ws`.**

If you don't specify, it will build a `ament-cmake` package by default. 

**3. Create the launch file for the `cartographer_node` with its configuration file.**

In Unit 2 of the ROS2 Navigation course, we learned how to create a launch file for the cartographer. 

**4. Launch the node using the launch file that you've just created, and create a map of the environment.**

In order to move the robot around the environment, you can use the keyboard teleop. To launch the keyboard teleop, just execute the following command:

In [ ]:
ros2 run teleop_twist_keyboard teleop_twist_keyboard

**IMPORTANT:** remember that in order to be able to move the robot with the keys, the terminal with the `teleop_keyboard` **has to have the focus**.

Also, remember to launch rviz and add the proper displays in order to visualize the map you are generating. You should get something like this in rviz:

<img src="images/RRL_nav_mapping_small.png"/>

Now, start moving the robot around to create the map. What's happening? Are you having difficulty generating a proper map? Is there anything strange in the mapping process? Do you know what could be the reason behind it?

What's happening here is the following:

* Your *TRAJECTORY_BUILDER_2D.max_range* parameter in the `cartographer.lua` configuration file is set to 0.5. This parameter sets the maximum range of the laser scanner that is used for map building. Since this is a small value, the robot can't get enough data to know where it is, so it may get lost. This will cause some strange issues during the mapping process, such as the robot readjusting its position incorrectly.
* In order to solve it, you should set the *TRAJECTORY_BUILDER_2D.max_range* parameter to some higher value, for instance, 3.5. This way, the robot will be able to get more data from the environemnt, so that it won't get lost. Now, you'll be able to finalize the mapping process correctly.

Once you modify this parameter and relaunch the mapping node, you should get something like this in rviz:

<img src="images/RRL_nav_mapping.png"/>

Now, you will be able to map the environment much more easily.

**5. Save the Map.**

Create a directory in your package named maps, and save the map files there. Your map image file should look something similar to this:

<img src="images/map.png"/>

**6. Create a launch file where you provide the map to the `map_server` node.**

As you've also seen in the course, the map you've just created will be used by other nodes (this means, it'll be used in further steps) in order to perform navigation.

Therefore, you'll need to create a launch file in order to provide the map. As you know, this is done through the `nav2_map_server` node.

Finally, launch this file and check that it's really providing the map.

<div>
    <h1 class="text-center">
        <span class="text-primary">Section 2</span>
        &nbsp;
        <span class="">Localization
    </h1>
</div>

In this section, you will localize the robot using the `amcl` node. This section has 3 steps:

1. Create a package called `project_localization` that will contain all of the files related to localization.
2. Create a **launch file** that will launch the `amcl` node and add the necessary parameters in order to properly configure the `amcl` node.
3. Launch the node and check that the robot properly localizes itself in the environment.

**1. Create a package called `project_localization`**

If you don't specify, it will build an `ament-cmake` package by default.

**2. Create a launch file for the `amcl` node**

In the localization section (Chapter 3) of this course, you saw how to create a launch file for the `amcl` node. You've also seen some of the most important parameters to set. So, in this step, you'll have to create a launch file for the amcl node and add the parameters you think you need to set.

Remember that before setting the amcl node parameters, you'll need to load the map created in Section 1. For that, you'll just need to include the launch you've created in Section 1 in the amcl node launch file in order to provide the map to other nodes.

**3. Launch the node and check that the TurtleBot3 robot correctly localizes itself in the map.**

Launch the node and check in rviz that the TurtleBot3 robot correctly localizes itself in the map.

In order to check that the localization works fine, move the robot around the environment and check that the particle cloud keeps getting smaller the more you move the robot. If you've done all of the previous steps correctly, you should see something like this in rviz:

Dispersed particles:

<img src="images/RRL_nav_amcl_spread.png"/>

Grouped particles:

<img src="images/RRL_nav_amcl_tight.png"/>

**4. Create the spots table.**

Once you've checked that localization is working fine, you'll need to create a table with 3 different spots in the environemnt. For each point, assign a tag (with the name of the spot) alongside its coordinates in the map.

These are the 3 spots that you'll have to register in the table:



You can access the coordinates of each position by looking into the topics that the `amcl` node publishes on. The only data that you really need to save are the position and orientation.

Next, you have a screenshot of the `/amcl_pose` topic:

<img src="images/amcl_pose.png"/>

Create a file named `spots.yaml` and write in the pose data that you've gotten from the 3 spots.

**5. Create a service that saves these spots into a file.**

In the previous section, you created a file with the coordinates of the 3 spots. But, the whole process was too cumbersome, don't you think? Why don't we create a ROS Service that does the work for us? Sounds like a good idea, right? Then, let's do it!

So, now you'll create a program that will do the following:

* It will launch a node named **`spot_recorder`**.
* This node will contain a service server named **`/save_spot`** that will take a string as input
* When this service is called, it will store the current coordinates of the robot (position and orientation values) with a label that will be the string provided on the service.
* When the end string is provided in the service call, the node will write all of the values stored into a file named **`spots.txt`**.
* Finally, the service will return a message indicating if the file has been saved correctly.

In order to achieve this, let's divide it into smaller parts.

#### 5.1. Service Message

First of all, you'll have to determine the kind of data you need for your service message.

* Determine what input data you need ( <span class="ign_blue">**request**</span> )
* Determine what data you want the service to return ( <span class="ign_blue">**response**</span> )

Next, see if there is an already-built message in the system that suits your needs. If there isn't, then you'll have to create your own custom message with the data you want.

If this is the case, then do the following. Inside the package you just created, create a new directory named **`/srv`**. Inside this directory, create a file named **`MyServiceMessage.srv`** that will contain the definition of your service message. 

This file could be something like this:

In [ ]:
# request
string label
---
#response
bool navigation_successfull
string message

Now, you'll have to modify the packages in your workspace in order for the system to find the service you have created.

When you've finished with the modifications, compile your package and check that your message has been correctly compiled and is ready to use. In order to check this, you can use the command:

In [ ]:
ros2 service list

#### 5.2. Service Code

Once your message is created, you're ready to use it in your service! So, let's write the code for our service. Create a file named **`spots_to_file.py`**. Inside this file, write the code necessary for your service.

#### 5.3. Launch File
Create a launch file for the node you've just created. You can also launch this node in the same launch file you've created to launch the mapping node. It's up to you.

#### 5.4. Test it
Using the keyboard teleop, move the robot to the 3 different spots shown above. At each one of these spots, perform a service call to the service you've just created. In the service call, provide the string with the name that you want to give to each spot.

For example, when you are at the bottom left corner of the lab, call your service like this:

In [ ]:
ros2 service call /record_spot "label: corner1"

Repeat this for each of the 3 spots. When finished, do one last service call, providing *end* as the string in order to create the file.

<div>
    <h1 class="text-center">
        <span class="text-primary">Section 3</span>
        &nbsp;
        <span class="">Path Planning System
    </h1>
</div>

In this section, you will create a path planning system using `Nav2` in order to navigate autonomously. This section has 4 actions for you to take:

* Create a package called **`project_path_planning`** that will contain all of the files related to Path Planning.
* Create a **launch file** that will launch all of the nodes required for path planning.
* Create the necessary parameter files in order to properly configure the `controller_server`, `planner_server`, `recoveries_server` and `bt_navigator` nodes.
* Launch the node and check that everything works as expected.

**1. Create a package named `project_path_planning`.**

**2. Create the launch file for the `controller_server`, `planner_server`, `behavior_server` and `bt_navigator` nodes.**

Now we need to add the following nodes to the launch file, created in the previous step:
* controller_server
* planner_server
* behavior_server
* bt_navigator

**3. Create the `project_params.yaml` file**

**4. Run launch file and navigate with `rviz2`**

Run the launch file and send navigation goals to the robot in `rviz2` to verify everything is working as expected.

<div>
    <h1 class="text-center">
        <span class="text-primary">Section 4</span>
        &nbsp;
        <span class="">Create ROS 2 node that interacts with Nav2
    </h1>
</div>

By this point, you have a working Navigation system on your robot. Now the next step is to make programs that interact with Nav2. 

In this step, we will create programs that will give commands to the robot. 

1. Create a spot recorder script.
2. Format the parameter file.
3. Create a script that makes the robot move to the given spot. 


**1. Create a spot recorder script.**

Earlier in this project, we created a map of the environment, but still we don't know the coordinates of any location on the map. 

To get to know the location of a point on the map, we will create a script that will save the location of a marked point from Rviz to the YAML file.

For that, we will subscribe to the `/initialpose` topic so that we can get the position and orientation of the robot at the spot. 

There are 3 spots you need to record for this project:

Fetch first corner (**`label: corner1`**):

<img src="images/corner1.png"/>

Fetch second corner (**`label: corner2`**):

<img src="images/corner2.png"/>

Fetch pedestrian cross (**`label: pedestrian`**):

<img src="images/pedestrian.png"/>

To record, you should start only the `map_server` node and `rviz2`, then select `2D_Pose_Estimate` to mark the location.


**2. Format the parameter file.**

In the step above, we have created a `.yaml` file that can be used to pass parameters to the `ros-parameter-server`, but we need to format it a bit according to the required minimal structure for ROS2 before we can use it as a parameter file. 

You need to add `node_name` & `ros__parameters` to the file. 

You need to update the name of the spot 
<div class="bg-info text-center">
    - Notes -
</div> 

* Here is the structure of the ROS parameter file. 

In [ ]:
node_1:
  ros__parameters:
    parameter_name: "abc"
node_2:
  ros__parameters:
    nested_parameter: 
        parameter_name: 45.2
node_3:
  ros__parameters:
    parameter_name: 45
    nested_parameter:
        parameter_name: value

**3. Create a script that moves the robot to a given spot.**

In this step, we will create a script that will take input from the `spot-list` & `spot-name`, so it can make the robot move to the desired spot on the map. 

Here we will make the action client for the `NavigateToPose` action, which will read the point location from the `spot-list.yaml` and move the robot to the desired spot, which is given as a parameter `spot-name`.


<div class="bg-info text-center">
    - Notes -
</div> 

* The node name should be the same, i.e. `move_to_spot`, in the parameter file and in the action client.

* To run this file, you need to add them to `setup.py`

<div class="bg-info text-center">
    - End of Notes -
</div> 

Run this command to launch the action client to move to your desired pose. 

In [ ]:
ros2 run nav2_project go_to_pose --ros-args --params-file spot-list.yaml -p spot_name:=corner1

<div>
    <h1 class="text-center">
        <span class="text-primary">Section 5</span>
        &nbsp;
        <span class="">Book a date to present the rosject on YouTube
    </h1>
</div>

### Presentation

An important part of your learning is to show others what you have learned. To get the most out of this course, you should deliver a live presentation of your rosject. 

If you want to take this brave step, contact us at info@theconstructsim.com to request a presentation and we will agree on the day and time.

**You will have to:**

1. Prepare your presentation. Explain your rosject in 20 minutes, how you solved it and how it works.
2. On the day agreed, you will need to have a camera and mic ready.
3. We will contact you on the day and prepare the broadcast. We will handle everything, you only need to be ready to present.

The event will broadcast on YouTube so anyone in the world can attend and watch your presentation!

<div>
    <h1 class="text-center">
        <span class="text-primary">Appendix</span>
        &nbsp;
        <span class="">Connect to the Real Robot
    </h1>
</div>

Once you know the basics of the operation with the simulation, it's time to use the real TurtleBot3 robot.

### Step 1: Book a Session
In the main dashboard, you can book a session by clicking on this icon:

<img src="images/reallab.png"/>

A booking page will appear where you can do two things:
* Check your bookings
<img src="images/bookassession.png"/>
* Make a new booking
<img src="images/bookings.png"/>
You can also click on the **BOOK A SESSION** button to make a new one, which will take you to a menu, where you will select:
* The **Type of Robot** you want to book. For the moment, only TurtleBot3 is available.
* The Date and Time for your reservation.
<img src="images/make_reservation_2.png"/>

The **date and times shown are the ones available.** They come in **25 minute slots.** There is also a limitation on the number of bookings per week a user can make. This depends on your particular license and subscription.
<img src="images/make_reservation_3.png"/>
<img src="images/make_reservation_4.png"/>

### Step 2: Launch the RoBox-TurtleBot3 rosject

To have the best experience, you need to launch the rosject with the simulation of TurtleBot3. Chances are, if you're reading this, you've already launched it.

When **one hour or less** is remaining until your booking session, a warning icon will be shown in **the bottom bar** of The Construct. This means that you have less than an hour to get ready for your session. You will also see the remaining time until your session.

<img src="images/booking_warnings1.png"/>

In case you are inside a rosject, the icon will appear at the bottom line of icons, like in the following image.

<img src="images/warning_4.png"/>

The orange dot will turn into a **red dot** when your booking session has started.

<img src="images/warning_2.png"/>

**Note:** When you are inside your booking, the streaming icon will appear on the desktop inside the rosject:
<img src="images/stream_icon.png"/>
<img src="images/RRL_stream.png"/>

### Step 3: Turn on the RoBoX Connection

Once you are inside the rosject and in your **booked time**, a toggle for **turning on the connection** will appear.

**WARNING: Nothing will appear unless you have a booking. So, if you didn't make one in the dashboard, feel free to do so.**

Now you just have to click on the toggle to connect the desktop environment to the remote lab. This will start the connection process, which lasts for a few seconds. 

Once the connection process is finished, the **red dot will change to green** if successfully connected. Now **any terminal you open will show the topics and services of the real robot**.

<img src="images/connect_robox.gif"/>

Now you should be able to do the following command, getting a list of topics similar to this. It could take about 30 seconds to appear:

In [ ]:
ros2 topic list

**Note:** Take into account that you **will not be able to push git changes when connected to RoBoX because there is no internet connection.** So, to push any changes to your remote gits, you will have to turn off the connection.

You are now connected to the robot! Let's try moving the robot around and seeing the laser and camera in rviz.

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p>
</th>
</tr>
</table>

Move the robot around: 

In [ ]:
ros2 run teleop_twist_keyboard teleop_twist_keyboard

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #2</p>
</th>
</tr>
</table>

In [ ]:
rviz2

You can then add an image and scan elements in the **base_link** frame, which will look something like this:

<img src="images/RRL_scan_cam.png"/>


## Ready for your programs!

If you have reached this point, it means that you have a proper connection with the remote real robot lab. **Now you can launch the programs of your project and see the results on a real robot**.